In [ ]:
import pandas as pd
import numpy as np
import copy

In [ ]:
df = pd.read_excel('HCP_Data_KDAG_Hackathon.xlsx')
df.drop(columns=['Vouchers_Dropped','Speaker_Programs_Attended'],inplace=True)
df['DateTime'] = df['Time_Period'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
df.drop(columns=['Time_Period'],inplace=True)
p = list(df['Physician_ID'].unique())
x =df.groupby(['Physician_ID'])
emp = []
for i in p:
    if (x.get_group(i)['Brand_Rx']==0.0).all():
        emp.append(i)

## Algorithm  for making recommendations (Task -1)

In [ ]:
idf = []
recomm = []
k = 0

In [ ]:
for i in p:
    
    idf.append(i)
    a = df[df['Physician_ID']==i]
    a = copy.deepcopy(a)
    call = a['Sales_Rep_Calls'].sum()
    s = list(a['Physician_Segment'].unique())
    l = ['Sales_Rep_Calls','Samples_Dropped','Emails_Delivered']
    a1 = a.loc[a['Brand_Rx'] !=0]
    
    va = [a1['Sales_Rep_Calls'].sum(),a1['Samples_Dropped'].sum(),a1['Emails_Delivered'].sum()]
    data = pd.DataFrame(data=va, index=l, columns=["score"]).sort_values(by = "score", ascending=False)
    if data.index[0] == 'Sales_Rep_Calls':
        if s[0] == '1-High':
            if call>48:
                if data.index[1] == 'Samples_Dropped':
                    if k>2500:
                        recomm.append(data.index[2])
                    else:
                        recomm.append(data.index[1])
                        k+=1
                elif data.index[1] == 'Emails_Delivered':
                    recomm.append(data.index[1])
            else:
                recomm.append(data.index[0])
    
        elif s[0] == '2-Medium':
            if call>24:
                if data.index[1] == 'Samples_Dropped':
                    if k>2500:
                        recomm.append(data.index[2])
                    else:
                        recomm.append(data.index[1])
                        k+=1
                elif data.index[1] == 'Emails_Delivered':
                    recomm.append(data.index[1])
            else:
                recomm.append(data.index[0])
                
    
        elif s[0] == '3-Low':
            if call>12:
                if data.index[1] == 'Samples_Dropped':
                    if k>2500:
                        recomm.append(data.index[2])
                    else:
                        recomm.append(data.index[1])
                        k+=1
                elif data.index[1] == 'Emails_Delivered':
                    recomm.append(data.index[1])
            else:
                recomm.append(data.index[0])
                
                
    elif data.index[0] == 'Samples_Dropped':
        
        if k>2500:
            if data.index[1] == 'Sales_Rep_Calls':
                if s[0] == '1-High':
                    if call>48:
                        recomm.append(data.index[2])
                    else:
                        recomm.append(data.index[1])
                elif s[0] == '2-Medium':
                    if call>24:
                        recomm.append(data.index[2])
                    else:
                        recomm.append(data.index[1])
                elif s[0] == '3-Low':
                    if call>12:
                        recomm.append(data.index[2])
                    else:
                        recomm.append(data.index[1])
            elif data.index[1] == 'Emails_Delivered':
                recomm.append(data.index[1])
        else:
            recomm.append(data.index[0])
            k+=1
        
    elif data.index[0] == 'Emails_Delivered':
        recomm.append(data.index[0]) 

In [ ]:
# Result dataframe for task1 
task1 = {'Physician_ID':idf,'Channel':recomm}
task1['Channel'] = task1['Channel'].map({'Emails_Delivered': 2, 'Sales_Rep_Calls': 0,'Samples_Dropped':1})
task1 = pd.DataFrame.from_dict(task1)

In [ ]:
#Result dataframe to CSV for task1
task1.to_csv('task1.csv',index=False)

## Model for making Brand_rx Predictions(Task -2)
## The model Parameters have been obtained after hyperparameter tuning and cross validation.

In [ ]:
#Install lightgbm if already not installed.
pip install lightgbm

In [ ]:
import lightgbm as lgb

In [ ]:
import pandas as pd

In [ ]:
ids=[]
pred=[]

In [ ]:
v = ['2019-01-04', '2019-01-11', '2019-01-18', '2019-01-25','2019-02-01', '2019-02-08', '2019-02-15', '2019-02-22','2019-03-01', '2019-03-08', '2019-03-15', '2019-03-22','2019-03-29', '2019-04-05', '2019-04-12', '2019-04-19','2019-04-26', '2019-05-03', '2019-05-10', '2019-05-17','2019-05-24', '2019-05-31', '2019-06-07', '2019-06-14','2019-06-21', '2019-06-28', '2019-07-05', '2019-07-12','2019-07-19', '2019-07-26', '2019-08-02', '2019-08-09','2019-08-16', '2019-08-23', '2019-08-30', '2019-09-06','2019-09-13', '2019-09-20', '2019-09-27', '2019-10-04','2019-10-11', '2019-10-18', '2019-10-25', '2019-11-01', '2019-11-08', '2019-11-15', '2019-11-22', '2019-11-29','2019-12-06', '2019-12-13', '2019-12-20', '2019-12-27','2020-01-03', '2020-01-10', '2020-01-17', '2020-01-24']
b = ['2019-01-11', '2019-01-18', '2019-01-25','2019-02-01', '2019-02-08', '2019-02-15', '2019-02-22','2019-03-01', '2019-03-08', '2019-03-15', '2019-03-22','2019-03-29', '2019-04-05', '2019-04-12', '2019-04-19','2019-04-26', '2019-05-03', '2019-05-10', '2019-05-17','2019-05-24', '2019-05-31', '2019-06-07', '2019-06-14','2019-06-21', '2019-06-28', '2019-07-05', '2019-07-12','2019-07-19', '2019-07-26', '2019-08-02', '2019-08-09','2019-08-16', '2019-08-23', '2019-08-30', '2019-09-06','2019-09-13', '2019-09-20', '2019-09-27', '2019-10-04','2019-10-11', '2019-10-18', '2019-10-25', '2019-11-01', '2019-11-08', '2019-11-15', '2019-11-22', '2019-11-29','2019-12-06', '2019-12-13', '2019-12-20', '2019-12-27','2020-01-03', '2020-01-10', '2020-01-17', '2020-01-24','2020-01-31']

In [ ]:
for i in p:
    if i in emp:
        ids.append(i)
        pred.append(0.0)
    else:
        ids.append(i)
        original = df[df['Physician_ID']==i]
        #ka = copy.deepcopy(ka)
        sub = original[['DateTime','Brand_Rx']]
        expanded = sub.set_index('DateTime').resample('D').ffill().reset_index()
        sub = sub.set_index('DateTime')
        br = []
        for ii,jj in zip(v,b):
            c= np.linspace(sub.loc[ii]['Brand_Rx'],sub.loc[jj]['Brand_Rx'],num=8)
            c.tolist()
            br.extend(c[:-1])
        br.append(sub['Brand_Rx'][-1])
        expanded['brand'] = br
        expanded['brand_lag1']=expanded['brand'].shift(1)
        expanded['brand_lag2']=expanded['brand'].shift(2)
        expanded['brand_lag3']=expanded['brand'].shift(3)
        expanded['brand_lag4']=expanded['brand'].shift(4)
        expanded['brand_lag5']=expanded['brand'].shift(5)
        expanded['brand_lag6']=expanded['brand'].shift(6)
        expanded['brand_lag7']=expanded['brand'].shift(7)
        expanded.fillna(0.0,inplace=True)
        FEATURES = ['brand_lag1','brand_lag2','brand_lag3','brand_lag4','brand_lag5','brand_lag6','brand_lag7']
        TARGET=['brand']
        X_train = expanded[FEATURES]
        y_train = expanded[TARGET]
        d_train=lgb.Dataset(X_train,label=y_train)
        lgb_params = {'learning_rate': 0.000001, 'boosting_type':'gbdt', 'metric':'rmse','num_leaves':100, 'max_depth':5}
        model = lgb.train(lgb_params,d_train)
        pre_vales1 = [expanded['brand'].iloc[-1],expanded['brand_lag1'].iloc[-1],expanded['brand_lag2'].iloc[-1],expanded['brand_lag3'].iloc[-1],expanded['brand_lag4'].iloc[-1],expanded['brand_lag5'].iloc[-1],expanded['brand_lag6'].iloc[-1]]
        pred_acul1 = model.predict(np.asarray([pre_vales1]))
        pred.append(pred_acul1[0])

In [ ]:
# Result dataframe for task2
task2 = {'Physician_ID':ids,'Expected_TRx':pred}
task2 = pd.DataFrame.from_dict(task2)

In [ ]:
#Result dataframe to CSV for task2
task2.to_csv('task2.csv',index=False)